In [19]:
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np

# PREGATIREA DATELOR
print("--- IncArcare Si Diagnosticare Date ---")
try:
    df = pd.read_csv('Prices.csv')
    df.columns = df.columns.str.strip().str.lower()


    col_price = 'price'
    col_speed = 'speed'

    col_hd = None
    possible_hd = ['hd', 'hdd', 'harddrive', 'ads', 'harddisc']
    for candidate in possible_hd:
        if candidate in df.columns:
            col_hd = candidate
            break

    if not col_hd:
        raise KeyError(f"Nu s-a gasit coloana pentru HDD. Variantele cautate: {possible_hd}")


    y_data = df[col_price].values
    x1_data = df[col_speed].values
    x2_data = np.log(df[col_hd].values)

    if 'premium' in df.columns:
        if df['premium'].dtype == 'object':
            x3_data = pd.get_dummies(df['premium'], drop_first=True).iloc[:, 0].values
        else:
            x3_data = df['premium'].values
    else:
        print("Atentie: Coloana 'premium' lipseste. Se folosesc date random.")
        x3_data = np.random.randint(0, 2, size=len(y_data))

    print("Datele au fost incarcate cu succes!\n")

except Exception as e:
    print(f"EROARE LA INCARCARE: {e}")
    np.random.seed(42)
    N = 500
    x1_data = np.random.randint(25, 100, N)
    x2_data = np.log(np.random.randint(80, 2000, N))
    x3_data = np.random.randint(0, 2, N)
    y_data = 1500 + 10 * x1_data + 200 * x2_data + 500 * x3_data + np.random.normal(0, 100, N)
    print("--> Se folosesc date sintetice.\n")

--- IncArcare Si Diagnosticare Date ---
Datele au fost incarcate cu succes!



Punctul a)

In [13]:

with pm.Model() as model:
    x1_shared = pm.Data('x1_shared', x1_data)
    x2_shared = pm.Data('x2_shared', x2_data)

    alpha = pm.Normal('alpha', mu=np.mean(y_data), sigma=1000)
    beta1 = pm.Normal('beta1', mu=0, sigma=100)
    beta2 = pm.Normal('beta2', mu=0, sigma=100)
    sigma = pm.HalfCauchy('sigma', beta=100)

    mu = pm.Deterministic('mu', alpha + beta1 * x1_shared + beta2 * x2_shared)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y_data)

    idata = pm.sample(2000, tune=2000, return_inferencedata=True, random_seed=42, progressbar=True)

print("Esantionare completa.\n")

Output()

Esantionare completa.



Punctul b)

In [14]:
print("--- Punctul b) 95% HDI pentru beta1 si beta2 ---")
summary_beta = az.summary(idata, var_names=['beta1', 'beta2'], hdi_prob=0.95)
print(summary_beta[['mean', 'sd', 'hdi_2.5%', 'hdi_97.5%']])
print("\n")

--- Punctul b) 95% HDI pentru beta1 si beta2 ---
          mean      sd  hdi_2.5%  hdi_97.5%
beta1    4.725   1.241     2.362      7.175
beta2  291.627  36.329   223.101    364.164




Punctul c)

In [15]:
beta1_low = summary_beta.loc['beta1', 'hdi_2.5%']
beta1_high = summary_beta.loc['beta1', 'hdi_97.5%']

if beta1_low > 0:
    print(f"-> Frecventa procesorului (beta1) ESTE un predictor util (interval pozitiv).")
elif beta1_high < 0:
    print(f"-> Frecventa procesorului (beta1) ESTE un predictor util (interval negativ).")
else:
    print("-> Frecventa procesorului NU pare a fi un predictor util (intervalul contine 0).")
print("\n")

-> Frecventa procesorului (beta1) ESTE un predictor util (interval pozitiv).




Punctul d)

In [16]:

new_speed = 33
new_hd_log = np.log(540)

with model:
    pm.set_data({
        'x1_shared': [new_speed],
        'x2_shared': [new_hd_log]
    })

    post = idata.posterior

    mu_calculated_xr = post['alpha'] + post['beta1'] * new_speed + post['beta2'] * new_hd_log

    mu_samples_np = mu_calculated_xr.values.flatten()

    mu_hdi = az.hdi(mu_samples_np, hdi_prob=0.90)

print(f"Pentru Speed=33 si HD=540:")
print(f"Valoarea medie asteptata (mu) 90% HDI: [{mu_hdi[0]:.2f}, {mu_hdi[1]:.2f}]\n")

Pentru Speed=33 si HD=540:
Valoarea medie asteptata (mu) 90% HDI: [2210.83, 2340.12]



Punctul e)

In [17]:

with model:
    try:
        post_pred = pm.sample_posterior_predictive(idata, predictions=True, random_seed=42, progressbar=False)
    except TypeError:
        post_pred = pm.sample_posterior_predictive(idata, random_seed=42, progressbar=False)

if isinstance(post_pred, az.InferenceData):
    if hasattr(post_pred, 'predictions') and 'y_obs' in post_pred.predictions:
        y_samples_xr = post_pred.predictions['y_obs']
    elif hasattr(post_pred, 'posterior_predictive') and 'y_obs' in post_pred.posterior_predictive:
        y_samples_xr = post_pred.posterior_predictive['y_obs']
    else:
        raise ValueError("Nu s-au gasit predictiile in obiectul InferenceData.")
else:
    y_samples_xr = post_pred['y_obs']

if hasattr(y_samples_xr, 'values'):
    y_samples_np = y_samples_xr.values.flatten()
else:
    y_samples_np = np.array(y_samples_xr).flatten()

y_hdi = az.hdi(y_samples_np, hdi_prob=0.90)

print(f"Pretul prezis (y_pred) 90% HDI: [{y_hdi[0]:.2f}, {y_hdi[1]:.2f}]\n")

Pretul prezis (y_pred) 90% HDI: [1447.40, 3107.72]



Bonus:
Faptul ca producatorul este Premium afectează pretul

In [18]:

with pm.Model() as model_bonus:
    alpha = pm.Normal('alpha', mu=np.mean(y_data), sigma=1000)
    beta1 = pm.Normal('beta1', mu=0, sigma=100)
    beta2 = pm.Normal('beta2', mu=0, sigma=100)
    beta3 = pm.Normal('beta3', mu=0, sigma=100)
    sigma = pm.HalfCauchy('sigma', beta=100)

    mu = alpha + beta1 * x1_data + beta2 * x2_data + beta3 * x3_data
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=y_data)

    idata_bonus = pm.sample(1000, tune=1000, return_inferencedata=True, progressbar=False, random_seed=42)

summary_bonus = az.summary(idata_bonus, var_names=['beta3'], hdi_prob=0.95)
print(summary_bonus[['mean', 'hdi_2.5%', 'hdi_97.5%']])

          mean  hdi_2.5%  hdi_97.5%
beta3 -194.379   -307.42    -81.905
